In [1]:
%pip install tiktoken
%pip install -qU langchain langchain_weaviate langchain-huggingface langchain-classic langchain-community sentence_transformers rank-llm==0.12.8
%pip install -qU weaviate-client
%pip install -qU langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.8/178.8 MB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip freeze | grep langchain

langchain==1.0.5
langchain-classic==1.0.0
langchain-community==0.4.1
langchain-core==1.0.4
langchain-huggingface==1.0.1
langchain-openai==1.0.2
langchain-text-splitters==1.0.0
langchain-weaviate==0.0.6


In [3]:
%pip freeze | grep rank

rank-llm==0.12.8


In [4]:
from google.colab import userdata
import getpass
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [5]:
import weaviate
from weaviate.classes.init import Auth

# Best practice: store your credentials in environment variables
weaviate_url = userdata.get("WEAVIATE_URL")
weaviate_api_key = userdata.get("WEAVIATE_API_KEY")

# Connect to Weaviate Cloud
weaviate_client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(weaviate_client.is_ready())  # Should print: `True`

True


In [6]:
from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", show_progress=True)

vectorstore = WeaviateVectorStore(client=weaviate_client, embedding=embeddings,
                                  index_name='PapadiamantisLangchain', text_key='text')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
vectorstore.similarity_search('φόνισσα', tenant='stories')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(metadata={'year': 1903.0, 'type': 'Διηγήματα', 'chapter': 'Not applied', 'title': 'Ἡ Φόνισσα', 'theme': 'Άγνωστη', 'chunk': 213.0}, page_content='Αὐτὴ διέκρινεν ἐναργῶς τὴν λέξιν τὴν ὁποίαν ἐπρόφερε τὸ λαλοῦν ἐκεῖνο νερόν: «Φόνισσα!… Φόνισσα!…»\nἈνετινάχθη φρίσσουσα, ἐξύπνησε, καὶ διετύπωσε πρὸς ἑαυτήν, ὡς εἰς παραμίλημα πυρετοῦ, μίαν ἀλλόκοτον ἐρώτησιν: «Τάχα τὸ αἷμα τὸ πνιγμένο φωνάζει, ὅπως καὶ τὸ αἷμα ποὺ χύθηκε;»\nΕἶτα εὐθὺς συνῆλθεν εἰς ἑαυτήν, ἐδοκίμασε πάλιν νὰ προφέρῃ τῆς προσευχῆς τὰ καταπραϋντικὰ λόγια: «Κύριε Ἰησοῦ…» Τὴν ἰδίαν στιγμὴν ἀνεπόλησε τὰ λησμονημένα λόγια ἑνὸς τροπαρίου, τὸ ὁποῖον εἶχεν ἀκούσει πολλὰς φορὰς εἰς τὴν νεότητά της νὰ ψάλλῃ ἕνας γέρων ἱερεύς: «Ἰησοῦ γλυκύτατε Χριστέ… Ἰησοῦ μακρόθυμε!»\nΤότε εὐθὺς τῆς ἦλθε πάλιν ὁ ὕπνος, βαθὺς καὶ διαρκέστερος. Καὶ τότε ὠνειρεύθη οἱονεὶ ὅτι ἐξαναέζη ὅλην τὴν περασμένην ζωήν της'),
 Document(metadata={'year': 1903.0, 'type': 'Διηγήματα', 'chapter': 'Not applied', 'title': 'Ἡ Φόνισσα', 'theme': 'Άγνωστη', 'chu

In [8]:
text_types = ["novels", "stories", "articles", "poems"]
base_retrievers = {}

for text in text_types:
    base_retrievers[text] = vectorstore.as_retriever(search_type="mmr", search_kwargs={"tenant": text, 'k': 10})

In [9]:
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)


compressor = RankLLMRerank(top_n=3, model="gpt", gpt_model="gpt-4o-mini")

compression_retrievers = {}

for text_type in text_types:
    #Compression retriever
    compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retrievers[text_type])

    compression_retrievers[text_type] = compression_retriever

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableBranch, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-2025-08-07", temperature=0)

In [11]:
import re

def classify_kind_from_query(query: str) -> str:
    query = query.lower()

    if re.search(r"\b(μυθιστόρημα|μυθιστορήματα)\b", query) or re.search(r"σαν μυθιστόρημα|σε μορφή μυθιστορήματος", query):
        return "novel"

    if re.search(r"\b(διήγημα|διηγήματα)\b", query) or re.search(r"σαν διήγημα|σε μορφή διηγήματος|μια μικρή ιστορία|σύντομη ιστορία", query):
        return "story"

    if re.search(r"\b(άρθρο|άρθρα)\b", query) or re.search(r"σαν άρθρο|με ύφος άρθρου|σε μορφή άρθρου", query):
        return "article"

    if re.search(r"\b(ποίημα|ποιήματα)\b", query) or re.search(r"γράψε.*ποίηση|γράψε.*ποίημα|σε μορφή ποιήματος|σαν ποίημα", query):
        return "poem"

    return "not specified"

In [12]:
retrievers = {
    "novel": compression_retrievers['novels'],
    "story": compression_retrievers['stories'],
    "article": compression_retrievers['articles'],
    "poem": compression_retrievers['poems'],
}

In [13]:
KIND_MAP = {
    "μυθιστόρημα": "novel",
    "διήγημα": "story",
    "άρθρο": "article",
    "ποίημα": "poem",
    "όλα": "all"
}

In [14]:
contextualize_instructions = "Χρησιμοποίησε τα παρακάτω αποσπάσματα κειμένου ως παράδειγμα:\n\n {context}."

prompt_with_context = ChatPromptTemplate.from_messages(
    [
        ("system", "Είσαι ένα συγγραφέας."),
        ("system", contextualize_instructions),
        ("user", "{question}")
    ]
)

In [15]:
prompt_without_context = ChatPromptTemplate.from_messages([
    ("system", "Είσαι ένα συγγραφέας."),
    MessagesPlaceholder(variable_name="history", optional=True),
    ("user", "{question}")
])

In [16]:
# Function to check if context exists
def context_exists(inputs: dict) -> bool:
    return bool(inputs.get("context"))

# Output parser
output_parser = StrOutputParser()

In [17]:
# Choose prompt based on context presence
dynamic_prompt = RunnableBranch(
    (context_exists, prompt_with_context),
    prompt_without_context
)

In [18]:
def format_context(documents):
    formated_documents = [f"Τίτλος: {doc.metadata['title']}\n{doc.page_content}" for doc in documents]
    return "\n\n".join(formated_documents)

def get_optional_context(inputs):
    if not inputs.get("use_context", True):
        return None
    kind = classify_kind_from_query(inputs["question"])
    if kind != "not specified":
        retriever = retrievers.get(kind)
        if retriever:
            docs = retriever.invoke(inputs["question"])
            return format_context(docs)
    else:
        while True:
            user_input = input("Επέλεξε το κειμενικό είδος για το οποίο θέλεις παραδείγματα: (μυθιστόρημα, διήγημα, άρθρο, ποίημα, όλα)")
            if user_input in KIND_MAP and user_input != "όλα":
                retriever = retrievers.get(KIND_MAP.get(user_input))
                if retriever:
                    docs = retriever.invoke(inputs["question"])
                    return format_context(docs)
            elif user_input in KIND_MAP and user_input == "όλα":
                results = []
                for _, retriever in retrievers.items():
                    docs = retriever.invoke(inputs["question"])
                    results.append(docs[0])
                return format_context(results)
            print("Δεν επέλεξες μία από τις επιλογές.")
    return None

In [19]:
get_optional_context({"use_context":True, "question": "Φόνισσα"})

Επέλεξε το κειμενικό είδος για το οποίο θέλεις παραδείγματα: (μυθιστόρημα, διήγημα, άρθρο, ποίημα, όλα)διήγημα


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.39s/it]


'Τίτλος: Ἡ Φόνισσα\nΑὐτὴ διέκρινεν ἐναργῶς τὴν λέξιν τὴν ὁποίαν ἐπρόφερε τὸ λαλοῦν ἐκεῖνο νερόν: «Φόνισσα!… Φόνισσα!…»\nἈνετινάχθη φρίσσουσα, ἐξύπνησε, καὶ διετύπωσε πρὸς ἑαυτήν, ὡς εἰς παραμίλημα πυρετοῦ, μίαν ἀλλόκοτον ἐρώτησιν: «Τάχα τὸ αἷμα τὸ πνιγμένο φωνάζει, ὅπως καὶ τὸ αἷμα ποὺ χύθηκε;»\nΕἶτα εὐθὺς συνῆλθεν εἰς ἑαυτήν, ἐδοκίμασε πάλιν νὰ προφέρῃ τῆς προσευχῆς τὰ καταπραϋντικὰ λόγια: «Κύριε Ἰησοῦ…» Τὴν ἰδίαν στιγμὴν ἀνεπόλησε τὰ λησμονημένα λόγια ἑνὸς τροπαρίου, τὸ ὁποῖον εἶχεν ἀκούσει πολλὰς φορὰς εἰς τὴν νεότητά της νὰ ψάλλῃ ἕνας γέρων ἱερεύς: «Ἰησοῦ γλυκύτατε Χριστέ… Ἰησοῦ μακρόθυμε!»\nΤότε εὐθὺς τῆς ἦλθε πάλιν ὁ ὕπνος, βαθὺς καὶ διαρκέστερος. Καὶ τότε ὠνειρεύθη οἱονεὶ ὅτι ἐξαναέζη ὅλην τὴν περασμένην ζωήν της\n\nΤίτλος: Ρεμβασμὸς τοῦ Δεκαπενταυγούστου\nΔὲν ἦτο πλέον φόβος νὰ γεννηθοῦν ἄλλα τέκνα. Ἡ Σινιώρα ἦτον ὑπερτεσσαρακοντοῦτις ἤδη.\nΤὴν ἑσπέραν ἐκείνην, τῆς 13 Αὐγούστου τοῦ ἔτους 186… ἐκάθητο μόνος, ὁλομόναχος, ἔξω τοῦ ναΐσκου, εἰς τὸ προαύλιον, ἔμπροσθεν τῆς καλύβης τὴ

In [20]:
chain = (
    {
        "question": lambda x: x["question"],
        "context": RunnableLambda(get_optional_context)
    }
    | dynamic_prompt
    | llm
    | output_parser
)

In [21]:
# Example usage:
question = "Γράψε μία παράγραφο για την Φόνισσα του Παπαδιαμάντη χρησιμοποιώντας καθαρεύουσα."

In [22]:
response = chain.invoke({
    "question": question,
    "use_context": False})
print(response)

Το περίφημον διήγημα «Η Φόνισσα» του Αλεξάνδρου Παπαδιαμάντη παριστά, διά ύφους σεμνού και ιεροπρεπούς, την τραγικήν περιπέτειαν της Φραγκογιαννούς, γυναικός πτωχής και καταπονημένης, ήτις, υπό το βάρος των ατέγκτων κοινωνικών εθίμων και της ανηλεούς πενίας, πείθεται ότι υπηρετεί ευεργεσίαν, θανατούσα νήπια, ίνα απαλλάξη τας μητέρας εκ του ζυγού της προικός και της ενδείας. Εν τω ασφυκτικώ κύκλω της νησιωτικής κοινωνίας της Σκιάθου, ο συγγραφεύς, μετά νηφαλιότητος και ευλαβούς στοχασμού, ανατέμνει την συνείδησιν της ηρωίδος, αποκαλύπτων βραδέως τον ζόφον και την δαιδαλώδη λογικήν μιας ψυχής ταλαντευομένης μεταξύ ενοχής, οίκτου και αυτοδικαιώσεως. Η λόγια γλώσσα, πυκνή και κατανυκτική, συνεργεί εις την ηθικήν ακτινοβολίαν του έργου, ενώ η δραματική λύσις, πλησίον θαλασσίου βράχου, υπογραμμίζει το τραγικόν μεθόριον μεταξύ θείας κρίσεως και ανθρωπίνης αδυναμίας.


In [23]:
response = chain.invoke({
    "question": question,
    "use_context": True  # or False to skip retriever
})

print(response)

Επέλεξε το κειμενικό είδος για το οποίο θέλεις παραδείγματα: (μυθιστόρημα, διήγημα, άρθρο, ποίημα, όλα)διήγημα


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


Ἡ «Φόνισσα» τοῦ Ἀλεξάνδρου Παπαδιαμάντη ἀποτελεῖ τραγικὴν ἀπεικόνισιν τῆς γυναικείας μοίρας καὶ τῆς ἀνελέητου ἀνάγκης ἐν τῇ κλειστῇ κοινωνίᾳ τῆς Σκιάθου. Κεντρικὸν πρόσωπον ἡ γραιὰ Χαδούλα ἡ Φραγκογιαννοῦ, γυνὴ ἐμπείρα κακῶν, ἡ ὁποία, ἔχουσα φαρμακερὰν φιλοσοφίαν, θεωρεῖ ὡς εὔσπλαγχνον ἔργον τὴν ἀπαλλαγὴν τῶν νεογνῶν θηλέων ἀπὸ τὰς μέλλουσας κακοπαθείας. Ἐν αὐλαῖς, χαράδροις καὶ θαλασσιναῖς ἐσοχαῖς ἡ ἄθλια περιπλανάται, ἄλλοτε ψιθυρίζουσα εὐχὰς, ἄλλοτε λογισμοὺς σκληροὺς κυοφοροῦσα, καὶ ἀθορύβως ἐπραγματοποίει τὴν φρικτὴν ἀπόφασίν της. Ἡ διήγησις, ἐμποτισμένη ἀπὸ εὐσέβειαν καὶ λαϊκὰς δοξασίας, ἀνάγει τὸ ἔγκλημα εἰς ἀναπότρεπτον μοῖραν, καὶ ἡ φύσις, κῦμα καὶ βράχος, ὁμιλεῖ ὡς μάρτυς ἀμείλικτος. Καθὼς τὰ ἴχνη τῆς στενεύουσι καὶ οἱ διώκοντες πλησιάζουσιν, ἡ Φραγκογιαννοῦ ζητεῖ καταφυγὴν πρὸς ἐξομολόγησιν ἐν Ἐρημητηρίῳ, ἀλλ᾽ ἡ θάλασσα τὴν ἀνακαλεῖ εἰς κρίσιν. Καὶ ἐκεῖ, ἐν τῷ μεταιχμίῳ μετανοίας καὶ ἀπωλείας, βυθιζομένη ἀπολείπει τὸν κόσμον, ἐξαγγέλλουσα δι᾽ αὐτῆς τῆς πτώσεως τὴν τραγικὴν ἀ